In [1]:
#imports needed for the code
import itertools
from scipy.stats import entropy
from collections import defaultdict, Counter
GUESSES = 10
WORD_FILE_all = "words.txt"
WORD_FILE = "correct_words.txt"

#generates all possible combinations of grey, green and yellow letters, represented as 0,1,2. combinations- we learned in lecture at one point
all_patterns = list(itertools.product([0,1,2],repeat =5))


In [2]:
#returns the difference between guess and real
def get_pattern(guess,real):
    wrong = [i for i,v in enumerate(guess) if v !=real[i]]
    counts = Counter(real[i] for i in wrong)
    pattern = [2,2,2,2,2]
    for i in wrong:
        v = guess[i]
        if counts[v]>0:
            pattern[i] = 1
            counts[v] -=1
        else:
            pattern[i] =0
    return tuple(pattern)

In [3]:
#return a list of combinations between words
def generate_valid(s,d):
    patterns = defaultdict(lambda: defaultdict(set))
    for word2 in d:
        patterns[s][get_pattern(s,word2)].add(word2)
    return dict(patterns)

In [10]:
orig_all_words = []

f = open('words.txt', "r")
for x in f:
    orig_all_words.append(x[0:5])
    
correct_words = []
f = open('correct_words.txt', "r")
for x in f:
    correct_words.append(x[0:5])
    
all_words = set(orig_all_words)

In [5]:
def calc_entropies(words, total_words):
    entropies = {}
    for word in words:
        counts = []
        for pattern in all_patterns:
            matches = generate_valid(word,all_words)[word][pattern]
            matches = matches.intersection(total_words)
            counts.append(len(matches))
        entropies[word]=entropy(counts)
    return entropies

In [11]:
#Try and pick something that has a higher matching, since the kernel takes forever to run it, like mathcing only an e with a guess of salet for bribe took 2 minutes to compile.
def wordle(mystery):
    MYSTERY_WORD = mystery
    STARTER_WORD = 'salet'
    all_words = set(orig_all_words)
    info = get_pattern(STARTER_WORD,MYSTERY_WORD)
    words = generate_valid(STARTER_WORD,all_words)
    words = words[STARTER_WORD][info]
    all_words = all_words.intersection(words)
#     print("The Mystery Word is:", MYSTERY_WORD)
#     print('The current guess is : ', STARTER_WORD)
#     print("The current matching status is: ", info)

    choices = generate_valid(STARTER_WORD,correct_words)[STARTER_WORD][info]
    entropies = calc_entropies(choices,all_words)
    for attempt in range(1,GUESSES):
#         print("The Mystery Word is:", MYSTERY_WORD)
    #took a shortcut b/c it took to long/ I cheated and looked through only the words in the right answer list that matched with our first guess. The only difference would be that choices would be all_words, but Cocalc can't handle 360*~13000.
        if len(all_words) ==1:
#             print("Wordle bot guesses:",all_words)
#             print("Guessed in this many rounds:", attempt)
            break
        guess_word = max(entropies, key=entropies.get)
        info = get_pattern(guess_word,MYSTERY_WORD)
#         print('The current guess is : ', guess_word)
#         print("The current matching status is: ", info)
        if guess_word.__eq__(MYSTERY_WORD):
#             print("Guessed in this many rounds:", attempt)
            break
        remaining = generate_valid(guess_word,all_words)[guess_word][info]
        all_words = all_words.intersection(remaining)
        entropies = calc_entropies(remaining,all_words)
    return (1+attempt)
   

In [12]:
print(wordle("pleat"))

KeyboardInterrupt: 